In [ ]:
import numpy as np
import sys
from scipy import linalg as LA

np.set_printoptions(threshold=sys.maxsize)

In [ ]:
%%time
#N: int # number of electrons
N = int(input("Enter valu of N : ") or "4")
t = 1
U = 0.5
def generate_basis_states():
    basis_array = []
    for i in range((2**(2*N))-1): # iterate over maximum possible number of states
        if bin(i).count("1") == N: # filter out possible states
            basis_array.append(i)
    # print("basis vector representation in decimal for N={0} ->".format(N), basis_array)
    # print("count of basis state for N = {0} is:".format(N), len(basis_array))
    return basis_array


basis_array = generate_basis_states()
print(np.size(basis_array))

In [ ]:
#def int_2_vectorized_state(m): # converts integer representation of a state into proper ket numpy array/vector


In [ ]:
import numpy as np

def int2bin(num):  # converts given integer 'n' iumnto binary of length 2N and return half parts as I1 and I2
    b_str = format(num, 'b').zfill(2*N) # add padding to make the length equal to 2N
    return np.fromiter(b_str, dtype=int) #convert str to array 
    #I1, I2 = b_str[:len(b_str)//2], b_str[len(b_str)//2:] # slicing into I1 and I2
    #return np.fromiter(I1, dtype=int), np.fromiter(I2, dtype=int)

   
# int2bin(15)

In [ ]:
import numpy as np
def Ht_operator_matrix(): # procedurally creates Ht operator matrix with #column=2N, #rows=4(N-1)
    a_up = np.zeros( (2*N-1, 2*N), dtype=int) 
    for i in range(2*N-1):
        a_up[i][i] = 1
        a_up[i][i+1] = -1
    a_up = np.delete(a_up,N-1,axis=0) #remove the (n-1) ie the central row from a_up matrix
    a_down = a_up* -1 
    return np.concatenate((a_up,a_down)) #concatenate a_down below a_up 
    
# c=Ht_operator_matrix()

In [ ]:
%%time
import numpy as np
def Ht_matrix_calculator():
    basis_states = generate_basis_states() # generate the basis state matrix for given N
    l = len(basis_states) # size of basis matrix for given N
    Ht_matrix = np.zeros((l, l), dtype=int) # final matrix containing l*l elements  to be returned    
    H = Ht_operator_matrix() # create Ht operator matrix

    for i in range(l):
        for j in range(l):
            phi_j = int2bin( basis_states[j] ) # pick jth vector from basis state matrix
            phi_i = int2bin( basis_states[i] )
            #operate H on phi_j. make a 2D (phi_j * height of H) matrix, add it to H element wise, call that H_phi_j
            H_phi_j = H + np.tile(phi_j, (4*(N-1),1) )
            # operate phi_i from left on H_phi_j , by just counting occurances of phi_i in H_phi_j, then append 1 to Ht_matrix[i][j] for each occurance
            occurance_count = sum((H_phi_j == phi_i).all(1))
            if occurance_count!=0:
                Ht_matrix[i][j]+=1
    return -t*Ht_matrix
            
glob_Ht = Ht_matrix_calculator()   
#print(a)      

In [ ]:
%%time
def Hu_matrix_calculator():
    basis_states = generate_basis_states() # generate basis state for a given N
    l = len(basis_states) #length of basis_state array
    Hu = np.zeros((l, l), dtype=int) # init empty Hu matrix 
    for i in range(l): 
        bin_I = int2bin(basis_states[i])  # convert each integer element of basis_state into binary number I
        I= np.hsplit(bin_I,2) #split I in half vertically and add both matrix
        sum_matrix = I[0]+I[1]
        #[Hu[i][i]= 1 for x in sum_matrix if x==2]
        # logic to check of a site contains both upspin and downspin, update Hu[i][i] it its true for ith element in binary state
        for x in sum_matrix:
            if(x==2):
                Hu[i][i]=1
    return U*Hu
    
glob_Hu = Hu_matrix_calculator()
#print(a)

In [ ]:
%%time
A = glob_Hu+glob_Ht
#print(A)
e_vals, e_vecs = LA.eig(A)


In [ ]:
#print(np.round(e_vals,2))
#print('*********************')
#print(np.round(e_vecs,2))


In [ ]:
A   = e_vals
pos = np.where(A == np.amin(A))
minima_psi = e_vecs[:,int(pos[0])]
#minima_psi = e_vecs[:,0]
#print(int(pos[0]))
np.size(basis_array) 

In [ ]:
mat1  = np.array([basis_array,np.square(np.transpose(minima_psi))]) 
#print(mat1)
b = mat1[:, np.argsort(mat1[-1, :])]
#print(b)
A   = b[1,:]
pos = np.where(A == np.amax(A))
#print(pos)
#pos[0][0]
print(b[0,pos[0][0]])
print(b[1,pos[0][0]])
max_spin_state = int2bin(int((b[0,pos[0][0]])))


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.bar(b[0,:],b[1,:])
plt.xlabel("Basis State in Binary")
plt.ylabel("Probability(coefficient squared)")
plt.savefig("Basis State Probabilities.jpeg")


from google.colab import files
#files.download('Basis State Probabilities.jpeg')

In [ ]:
max_spin_state